# AI Tutor Debugging Session

This notebook is designed for step-by-step debugging of the AI Tutor's core feedback loop. Each cell represents a single API call, allowing for careful inspection of the requests and responses.

In [1]:
import requests
import json
import time

# --- Configuration ---
BASE_URL = "http://127.0.0.1:8000"
USER_ID = f"debug_user_{int(time.time())}"
QUESTION_NUMBER = 3 # A question we know the answer to
INCORRECT_ANSWER = "1" # The 1-based index for 'δ (delta)'
CORRECT_ANSWER = "2" # The 1-based index for 'ε (epsilon)'

session = requests.Session()
hint_data = {} # To store hint response

print(f"Using User ID: {USER_ID}")

Using User ID: debug_user_1761461888


## Step 1: Create the User

In [2]:
payload = {"user_id": USER_ID}
response = session.post(f"{BASE_URL}/users/", json=payload)

print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 200
{
  "user_id": "debug_user_1761461888",
  "created_at": "2025-10-26T06:58:13.203879",
  "preferences": {
    "hint_style_preference": "adaptive",
    "intervention_preference": "manual"
  },
  "feedback_scores": {},
  "skill_mastery": [],
  "interaction_history": []
}


## Step 2: Set User Preferences

In [3]:
payload = {
    "intervention_preference": "proactive",
    "hint_style_preference": "adaptive"
}
response = session.put(f"{BASE_URL}/users/{USER_ID}/preferences/", json=payload)

print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 200
{
  "hint_style_preference": "adaptive",
  "intervention_preference": "proactive"
}


## Step 3: Submit an INCORRECT First Answer

In [4]:
payload = {
    "user_id": USER_ID,
    "question_number": QUESTION_NUMBER,
    "user_answer": INCORRECT_ANSWER
}
response = session.post(f"{BASE_URL}/answer/", json=payload)

print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status Code: 200
{
  "correct": false,
  "correct_answer": "2",
  "skill": "[LoRA-Quantization-Basics]",
  "intervention_needed": true,
  "current_mastery": 0.17575757575757575
}


## Step 4: Request a Hint

This is the most critical step. We need to inspect the full response body to see if `context` and `final_prompt` are being populated correctly.

In [5]:
payload = {
    "user_id": USER_ID,
    "question_number": QUESTION_NUMBER,
    "user_answer": INCORRECT_ANSWER
}
response = session.post(f"{BASE_URL}/hints/", json=payload)

print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

if response.ok:
    hint_data = response.json()

Status Code: 200
{
  "question_number": 3,
  "hint": "The document uses a Greek letter to represent the quantization error. Review the equations and surrounding text in section 2.2 to find the correct symbol.",
  "user_id": "debug_user_1761461888",
  "hint_style": "Conceptual",
  "pre_hint_mastery": 0.17575757575757575,
  "context": "Quantization-Aware Low-Rank Adaptation (QA-LoRA) \u2014 \nA Code + Concept Deep Dive \n \n1. Introduction & Motivation \nParameter-efficient fine-tuning (PEFT) methods like LoRA allow us to adapt large language \nmodels by inserting small low-rank adapters, freezing the base weights, and only training \nadapter components. QLoRA extends this by quantizing the base model (e.g. to 4 bits) and \nkeeping it frozen, while training LoRA adapters in higher precision (e.g. 16- or 32-bit) on top of \nit. \nOne challenge: quantizing the base model introduces quantization error (i.e. the quantized \nweight (\\hat W) deviates from the original full-precision weight (W

## Step 5: Submit a CORRECT Second Answer (with Hint Context)

In [6]:
if not hint_data:
    print("Cannot proceed: hint_data was not populated in the previous step.")
else:
    payload = {
        "user_id": USER_ID,
        "question_number": QUESTION_NUMBER,
        "user_answer": CORRECT_ANSWER,
        "hint_shown": True,
        "hint_style_used": hint_data.get("hint_style"),
        "hint_text": hint_data.get("hint"),
        "pre_hint_mastery": hint_data.get("pre_hint_mastery"),
        "feedback_rating": 5 # Assume a good hint
    }
    response = session.post(f"{BASE_URL}/answer/", json=payload)

    print(f"Status Code: {response.status_code}")
    print(json.dumps(response.json(), indent=2))

Status Code: 200
{
  "correct": true,
  "correct_answer": "2",
  "skill": "[LoRA-Quantization-Basics]",
  "intervention_needed": false,
  "current_mastery": 0.5662288930581614
}
